In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
# drive.mount('/content/drive')
import os
skipDataCreate=False
skipDataCreate=True
%cd /content/drive/MyDrive/Final Project/model
os.listdir()



Mounted at /content/drive
/content/drive/MyDrive/Final Project/model


['rnn.ipynb',
 'generator.py',
 '__pycache__',
 'writer.py',
 'resnet9.ipynb',
 'eli',
 'indrnn.py',
 'data',
 'model_rnn',
 'RNNV4.ipynb',
 'writer.ipynb',
 'log_Jun-18-2022_12-50-57.txt',
 'log_Jun-18-2022_12-54-54.txt',
 'log_Jun-18-2022_12-55-59.txt',
 'datacounter.ipynb',
 'log_Jun-18-2022_13-04-19.txt',
 'log_Jun-18-2022_13-04-41.txt',
 'log_Jun-18-2022_13-05-09.txt',
 'log_Jun-18-2022_13-05-45.txt',
 'log_Jun-18-2022_13-06-08.txt',
 'bestParams.txt']

In [2]:
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Israel /etc/localtime
!date

Sat Jun 18 16:11:13 IDT 2022


In [3]:
# # for colab only
if not skipDataCreate:
  # if writer:
  #   del writer
# !rm data
  import writer
  writer.resizeFactor=1 # set whether or not to resize the data (for pictures only)
  writer.how_many_orbits=20 # set the number of files to generate
  writer.generator.num_of_points=1000 # set the number of data rows in each file
  writer.csvWrite=True # write files as a csv only
  writer.oneFolder=True # put all files in one folder
  writer.mainFunc()
  del writer
  # writer.mainFunc()
else:
  print("Skipped")

# # print(os.getcwd())
datas=[]
for k in ["loc"]:
  for j in ['0','1','2','3']:
    for filename in os.listdir(f"./data/{k}/test/{j}"):
        if filename.endswith("csv"): 
            datas.append(f"./data/{k}/test/{j}/"+filename)
datas

Skipped


['./data/loc/test/0/data_44582fdf-f620-4ba0-9740-8f672fefc193.csv',
 './data/loc/test/0/data_5492c652-42a0-40ad-a30a-4154ced23b85.csv',
 './data/loc/test/0/data_25a9136e-7109-477c-af1d-d61e5165ac8b.csv',
 './data/loc/test/0/data_3aec1c79-aad2-422f-aa3f-b70c8c734eec.csv',
 './data/loc/test/0/data_b6f5c395-dcf5-480c-bb97-903aa1efe249.csv',
 './data/loc/test/0/data_fe1fcd93-5ff1-4f07-864f-3600b175dc5a.csv',
 './data/loc/test/0/data_f3ec9f86-01a4-4e76-b23f-a397e9cb812d.csv',
 './data/loc/test/0/data_00fa0bb6-3ba3-4d84-a8ea-8e2c3d974d86.csv',
 './data/loc/test/0/data_9d5a2144-1e3f-4ba8-a5a0-8b297fc016d7.csv',
 './data/loc/test/0/data_ea646f7a-a485-4ffa-8e35-6cffd62d75f1.csv',
 './data/loc/test/0/data_d8688b48-b6c5-4620-9065-7c9575855282.csv',
 './data/loc/test/1/data_4e2e3af2-b5e0-4c8d-a460-4854a328bb1c.csv',
 './data/loc/test/1/data_67c213a8-7b50-4692-8101-428a7f3cbe21.csv',
 './data/loc/test/1/data_61ad2271-483c-4c84-a887-0ede66770dd5.csv',
 './data/loc/test/1/data_16e1b974-afb7-400c-a6c5

In [4]:
# LSTM for international airline passengers problem with memory
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pdb; 

# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [5]:
def LSTM_Reg(dataset,train_size=0.67,batch_size=1,train_epochs=100,per_train_epochs=5,plot=False,look_back = 3,verbose=1,LSTM_nodes=1,Dense_nodes=1):
  # pre-prep the data
  scaler = MinMaxScaler(feature_range=(0, 1))
  dataset = scaler.fit_transform(dataset) # normalize
  # split into train and test sets
  train_size = int(len(dataset) * train_size)
  test_size = len(dataset) - train_size
  train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
  # reshape into X=t and Y=t+1
  trainX, trainY = create_dataset(train, look_back)
  testX, testY = create_dataset(test, look_back)
  # reshape input to be [samples, time steps, features]
  trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
  testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))
    
  model = Sequential()
  model.add(LSTM(LSTM_nodes, batch_input_shape=(batch_size, look_back, 1), stateful=True))
  model.add(Dense(Dense_nodes))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error', optimizer='adam')
  for i in range(train_epochs):
    if verbose:
      print("Superepoch %d of %d" % (i,train_epochs))
    model.fit(trainX, trainY, epochs=per_train_epochs, batch_size=batch_size, verbose=verbose, shuffle=False)
    model.reset_states()
  # make predictions
  trainPredict = model.predict(trainX, batch_size=batch_size)
  model.reset_states()
  testPredict = model.predict(testX, batch_size=batch_size)
  # invert predictions
  trainPredict = scaler.inverse_transform(trainPredict)
  trainY = scaler.inverse_transform([trainY])
  testPredict = scaler.inverse_transform(testPredict)
  testY = scaler.inverse_transform([testY])
  # calculate root mean squared error
  trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
  testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
  if verbose:
    print('Train Score: %.2f RMSE' % (trainScore))
    print('Test Score: %.2f RMSE' % (testScore))
  
  
  # shift train predictions for plotting

  if plot:
    # PLT
    trainPredictPlot = numpy.empty_like(dataset)
    trainPredictPlot[:, :] = numpy.nan
    trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
    # shift test predictions for plotting
    testPredictPlot = numpy.empty_like(dataset)
    testPredictPlot[:, :] = numpy.nan
    testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
    # plot baseline and predictions
    plt.plot(scaler.inverse_transform(dataset))
    plt.plot(trainPredictPlot)
    plt.plot(testPredictPlot)
    plt.show()
  return testScore

In [ ]:
# fix random seed for reproducibility
numpy.random.seed(7)
xyz=["y","x","z"]

bestParams=()
maxScore=0
verbose=0
try:
  with open("bestParams.txt") as BC:
    BC=BC.readlines()
    maxScore=float(BC[1])
    BC=BC[0][1:-2].split(', ')
    bestParams=tuple(map(float,BC))
    print("Score file loaded, score is:")
    print(maxScore)
    print("Best parameters found for now are the following:")
    print(BC)
except IOError:
  print("Score file not found, starting from 0")

from datetime import datetime
today = datetime.now()
d4 = today.strftime("%b-%d-%Y_%H-%M-%S")
logfile = open(f"log_{d4}.txt","a+")

# iterate through various parameters to find the best ones
for data in datas:
  for Dense_nodes in range(1,10,1):
    for LSTM_nodes in range(80,100,4):
      for per_train_epochs in range(1,9,2):
        for look_back in range(5,8):
          for train_epochs in range(6,10,1):
            for train_size in numpy.arange(0.65,0.95,0.1):        
              outerScore=0
              for j in range(1,4):
                curScore=0
                for i in range(1,4):
                  # load the dataset with given column
                  dataframe = read_csv(data, usecols=[i], engine='python')
                  dataset = dataframe.values
                  dataset = dataset.astype('float32')
                  # print(dataset==[])

                  if dataset.shape == (0,0):
                    continue
                  score = LSTM_Reg(dataset,per_train_epochs=per_train_epochs,look_back=look_back,train_epochs=train_epochs,train_size=float(train_size),verbose=verbose,Dense_nodes=Dense_nodes,LSTM_nodes=LSTM_nodes)
                  if type(score)==type(None):
                    score=0
                  if verbose:
                    print(f'{xyz[i-1]} score:\t{score}')
                  curScore += score          
                curScore/=3
                outerScore+=curScore
              outerScore/=3
              
              d4 = today.strftime("%b-%d-%Y_%H-%M-%S")
              logfile.writelines([f"{d4}\t|\t{outerScore}\t|\t{(per_train_epochs,look_back,train_epochs,train_size,Dense_nodes,LSTM_nodes)}\n"])
              if outerScore > maxScore:
                logfile.write(f"***\nCHANGE\n***")
                print(f'Old score ({maxScore}) => New Score ({outerScore})')
                maxScore=outerScore
                bestParams=(per_train_epochs,look_back,train_epochs,train_size,Dense_nodes,LSTM_nodes)
                with open('bestParams.txt', 'w') as writefile:
                  writefile.write(f"{bestParams}\n{maxScore}")

Score file loaded, score is:
0.34654186677144266
Best parameters found for now are the following:
['1', '7', '1', '0.9000000000000002', '1', '4']


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: FutureWarning: Defining useco

In [ ]:
# # print(f"best Params:\n{bestParams}")
# # print(f"best score:\n{maxScore}")
# with open('bestParams', 'w') as writefile:
#     writefile.write(f"{bestParams}\n{maxScore}")



In [ ]:
# !ls
# !ls -al /usr/share/zoneinfo/
logfile.close()

# New Section